# Data Download

## Install Modules

In [1]:
from __future__ import print_function
import time
import intrinio_sdk
from intrinio_sdk.rest import ApiException
import mysql.connector as sql
import pandas as pd
import DBcm
import requests
import unidecode #used to convert accented words

## Configuration

In [2]:
config = {
    "host": "127.0.0.1",
    "user": "root",
    "password": "root",
    "database": "STOCK_DB",
}
prod_ky = 'OmQ3ZDg2Mzc5MzJkYjEwOWRiNjJlYjE0YjkyMGZhM2Mz'
api_ky = prod_ky
intrinio_sdk.ApiClient().configuration.api_key['api_key'] = api_ky
company_api = intrinio_sdk.CompanyApi()

## Get Companies 

In [3]:
page_size = 10000 # float | The number of results to return (optional) (default to 100)
next_page = '' # str | Gets the next page of data from a previous API call (optional)

In [44]:
try:
    data = company_api.get_all_companies(page_size=page_size, next_page=next_page)
    companies_list = data.companies
except ApiException as e:
    print("Exception when calling CompanyApi->filter_companies: %s\n" % e)

In [32]:
for i in companies_list:
    with DBcm.UseDatabase(config) as cursor:
        _SQL = """insert into COMPANIES
                  (cik, company_id, lei, name, ticker)
                  values
                  (%s, %s, %s, %s, %s)""" 
        cursor.execute(_SQL, (i.cik, i.id, i.lei, i.name, i.ticker))

In [50]:
page_size = 10000 # float
next_page = 'MTIxOTk=' # str | Gets the next page of data

In [51]:
try:
    data = company_api.get_all_companies(page_size=page_size, next_page=next_page)
    companies_list2 = data.companies
except ApiException as e:
    print("Exception when calling CompanyApi->filter_companies: %s\n" % e)

In [53]:
for i in companies_list2:
    with DBcm.UseDatabase(config) as cursor:
        _SQL = """insert into COMPANIES
                  (cik, company_id, lei, name, ticker)
                  values
                  (%s, %s, %s, %s, %s)""" 
        cursor.execute(_SQL, (i.cik, i.id, i.lei, i.name, i.ticker))

There are totally 17,294 Companies listed in US Stock Exchange 

In [4]:
db_connection = sql.connect(host='127.0.0.1', database='STOCK_DB', user='root', password='root')
df = pd.read_sql('SELECT * FROM COMPANIES', con=db_connection)

In [4]:
df.shape

(17294, 6)

In [5]:
df.head()

,ID,CIK,COMPANY_ID,LEI,NAME,TICKER
0,5605,0000320193,com_NX6GzO,HWUPKR0MPOU8FGXBT394,Apple Inc,AAPL
1,5606,0000034088,com_2zvNzA,J3WHBG0MTS7O8ZVMDC91,Exxon Mobil Corp,XOM
2,5607,0001341439,com_0XLDXx,1Z4GXXU7ZHVWFCD8TV52,Oracle Corp,ORCL
3,5608,0000021344,com_VXWJgv,UWJKFUJFZ02DKWI3RY53,Coca-Cola Co,KO
4,5609,0000050863,com_NgYGzd,KNX4USFCNGPY45LOCE31,Intel Corp,INTC


In [6]:
df.tail()

,ID,CIK,COMPANY_ID,LEI,NAME,TICKER
17289,22894,0000889469,com_gxK0Ly,None,Blue Bird Corp,BLBD
17290,22895,0001618006,com_gkNKBy,None,CSOP Asset Management Ltd,HAHA
17291,22896,0001740332,com_gjm5Vy,None,Resideo Technologies Inc,REZI
17292,22897,0001735707,com_ylNR8X,None,Garrett Motion Inc,GTX
17293,22898,0001666071,com_y3Ao2X,549300JETJBH94EIPC66,Cardlytics Inc,CDLX


## Get News from Companies

In [12]:
#Loop thru dates for July Month
for i in range(1,32): #End period should be last day + 1. Start period is always 1
    
    curr_day = "{0:0=2d}".format(i)
    
    print('Processing for day - ' + curr_day + ' July 2019')

    #Define parameters
    d = {
        'start_date': '2019-07-' + curr_day,
        'start_time': '00:00:00',
        'end_date': '2019-07-' + curr_day,
        'end_time':'23:59:59',
        'page_size':'10000'
    }
    
    url = 'https://api.intrinio.com/news?api_key=' + prod_ky + '&'
    
    #Create URL
    for key, value in d.items():
        if key == 'page_size':
            url+= str(key + '=' + value)
        else:
            url+= str(key + '=' + value + '&')
    
    #Obtain the response
    r = requests.get(url = url)
    data = r.json()
    
    print('started loading table...')

    #load the data
    for j in data['data']:

        ticker = None
        figi_ticker = None
        figi = None
        title = None
        publication_date = None
        summary = None
        url = None

        if j['ticker'] != None:
            ticker = unidecode.unidecode(j['ticker'])
        if j['figi_ticker'] != None:
            figi_ticker = unidecode.unidecode(j['figi_ticker'])
        if j['figi'] != None:
            figi = unidecode.unidecode(j['figi'])
        if j['title'] != None:
            title = unidecode.unidecode(j['title'])
        if j['publication_date'] != None:
            publication_date = unidecode.unidecode(j['publication_date'])
        if j['summary'] != None:
            summary = unidecode.unidecode(j['summary'])
        if j['url'] != None:
            url = unidecode.unidecode(j['url'])

        with DBcm.UseDatabase(config) as cursor:
            _SQL = """insert into NEWS
                      (ticker, figi_ticker, figi, title, publication_date, summary, url)
                      values
                      (%s, %s, %s, %s, %s, %s, %s)"""
            cursor.execute(_SQL, (ticker, figi_ticker, figi, title, publication_date, summary, url))       
  

    print('Completed.')

print('Process Done.')

Processing for day - 01 July 2019
started loading table...
Completed.
Processing for day - 02 July 2019
started loading table...
Completed.
Processing for day - 03 July 2019
started loading table...
Completed.
Processing for day - 04 July 2019
started loading table...
Completed.
Processing for day - 05 July 2019
started loading table...
Completed.
Processing for day - 06 July 2019
started loading table...
Completed.
Processing for day - 07 July 2019
started loading table...
Completed.
Processing for day - 08 July 2019
started loading table...
Completed.
Processing for day - 09 July 2019
started loading table...
Completed.
Processing for day - 10 July 2019
started loading table...
Completed.
Processing for day - 11 July 2019
started loading table...
Completed.
Processing for day - 12 July 2019
started loading table...
Completed.
Processing for day - 13 July 2019
started loading table...
Completed.
Processing for day - 14 July 2019
started loading table...
Completed.
Processing for day -

## Get Stock data for the corresponding News Data

In [5]:
ticker_df = pd.read_sql('select distinct ticker from STOCK_DB.news', con=db_connection)

In [6]:
ticker_list = ticker_df.values.T.tolist()

In [7]:
ticker_list = ticker_list[0]

In [8]:
ticker_list.remove(None)

In [17]:
for n, ticker in enumerate(ticker_list):
    if ticker == 'CFDB':
        print(n,ticker)
#     if n > 2100:
#         print(n, ticker)

6894 CFDB


In [20]:
ticker_list[:5]

['TWTR', 'SLB', 'HYMLF', 'TTM', 'ADI']

In [18]:
#for ticker in ticker_list:
for ticker in ticker_list[6894:]:
        
    print('Processing for Ticker - ' + ticker)

#     ticker = 'WLAN'

    #Define parameters
    d = {
        'start_date': '2019-03-01',
        'end_date': '2019-03-31',
        'page_size':'50'
    }

    url = 'https://api-v2.intrinio.com/securities/' + ticker + '/prices?api_key=' + prod_ky + '&'

    #Create URL
    for key, value in d.items():
        if key == 'page_size':
            url+= str(key + '=' + value)
        else:
            url+= str(key + '=' + value + '&')
    
    #Obtain the response
    r = requests.get(url = url)
    data = r.json()
        
    #load the data
    try:
        for j in data['stock_prices']:

            with DBcm.UseDatabase(config) as cursor:
                _SQL = """insert into STOCK
                          (
                            security_id,
                            company_id,
                            stock_exchange_id,
                            name,
                            currency,
                            ticker,
                            composite_ticker,
                            figi,
                            composite_figi,
                            share_class_figi,
                            date,
                            intraperiod,
                            frequency,
                            open,
                            high,
                            low,
                            close,
                            volume,
                            adj_open,
                            adj_high,
                            adj_low,
                            adj_close,
                            adj_volume
                          )
                          values
                          (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s, %s, %s, %s, %s)"""
                cursor.execute(_SQL, (
                            data['security']['id'],
                            data['security']['company_id'],
                            data['security']['stock_exchange_id'],
                            data['security']['name'],
                            data['security']['currency'],
                            data['security']['ticker'],
                            data['security']['composite_ticker'],
                            data['security']['figi'],
                            data['security']['composite_figi'],
                            data['security']['share_class_figi'],
                            j['date'],
                            j['intraperiod'],
                            j['frequency'],
                            j['open'],
                            j['high'],
                            j['low'],
                            j['close'],
                            j['volume'],
                            j['adj_open'],
                            j['adj_high'],
                            j['adj_low'],
                            j['adj_close'],
                            j['adj_volume']
                        ))
                
    except KeyError as e:
        print(data['error'])
        
print('Process Done.')    

Processing for Ticker - CFDB
Processing for Ticker - IDFB
Processing for Ticker - JMIA
Processing for Ticker - TSK
Processing for Ticker - PMPG
Processing for Ticker - SKYL
Processing for Ticker - MYDX
Processing for Ticker - GW
Processing for Ticker - AQXP
Processing for Ticker - CVVT
Processing for Ticker - CCTC
Processing for Ticker - TOTA
Processing for Ticker - ALNPF
Processing for Ticker - EUO
Processing for Ticker - ACTTU
Processing for Ticker - BXLC
Processing for Ticker - CRD.B
Processing for Ticker - CLC
Processing for Ticker - FLTX
Processing for Ticker - DNJR
Processing for Ticker - EFBI
Processing for Ticker - BCM
Processing for Ticker - CCOM
Processing for Ticker - TRUU
Processing for Ticker - MYMX
Processing for Ticker - REMI
Processing for Ticker - YUANF
Processing for Ticker - CHNR
Processing for Ticker - OMNT
Processing for Ticker - NSAV
Processing for Ticker - YETI
Processing for Ticker - IMTE
Processing for Ticker - PSTL
Processing for Ticker - QUCPF
Processing for 

Processing for Ticker - DBGF
Processing for Ticker - MCTC
Processing for Ticker - AAAU
Processing for Ticker - VCAN
Processing for Ticker - PLBN
Processing for Ticker - LINX
Processing for Ticker - CCT
Processing for Ticker - GAHC
Processing for Ticker - PRXL
Processing for Ticker - EXAC
Processing for Ticker - AHFP
Processing for Ticker - NDYN
Processing for Ticker - TMCX
Processing for Ticker - AWSL
Processing for Ticker - ANFC
Processing for Ticker - LAYN
Processing for Ticker - TMBXF
Processing for Ticker - CMBM
Processing for Ticker - YGE
Processing for Ticker - AMCC
Processing for Ticker - FRP
Processing for Ticker - BONZ
Processing for Ticker - CHNGQ
Processing for Ticker - TPCA
Processing for Ticker - ROFO
Processing for Ticker - PRMI
Processing for Ticker - CSH
Processing for Ticker - LYBC
Processing for Ticker - TE
Processing for Ticker - IGMB
Processing for Ticker - SKYF
Processing for Ticker - CBKW
Processing for Ticker - DLTZ
Processing for Ticker - BOLT
Processing for Tic